# This notebook normalizes landmarks in VD_FEATURES_L2 using the mean measurement from points 0 to 16 as a reference.

# Import dependencies

In [1]:
import pandas as pd, numpy as np, os, sys, glob

# Import library with current code functions
sys.path.append(os.path.join("..", "..", "lib"))
import video_adjuster_functions as vid_adj_fun

# Functions

In [2]:
baseDir = os.path.join('..', 'Dataset')

# Define the name of output csv file
output_measure_name = 'VD_FEATURES_L2_NORMALIZED.CSV'

# Search path of VD_INFO files
MAIN_LIST_INFO = []

# File_name to find
file_name_fd = 'VD_INFO'

FOLDER_PATH = os.path.join(baseDir, '**', file_name_fd)
MAIN_LIST_INFO = sorted(glob.iglob(FOLDER_PATH, recursive=True ))

# VD_FEATURE_L2 Path
VD_FEATURES_FILE_PATH = 'VD_FEATURES_L2.CSV'

In [3]:
ptr = 1
for current_path_location in MAIN_LIST_INFO:
    
    # Read VD_FEATURE
    path_dir = os.path.dirname(current_path_location)
    vd_feature_path = os.path.join(path_dir, VD_FEATURES_FILE_PATH)
    video_feature_rest = pd.read_csv(vd_feature_path)
    
    # Remove the Unnamed columns
    video_feature_rest.drop(columns=['Unnamed: 0'], inplace=True)

    video_feature_rest_drop_na = video_feature_rest.dropna()

    # Get values of frame_seq and video_id
    face_horz = vid_adj_fun.measure_two_points_from_csv_ext (video_feature_rest_drop_na, 1-1, 17-1, 'face_horz')

    # Normalize taking as reference the mean of measure from points 0 to 16
    dist_base = face_horz.mean().iloc[0]
    
    VIDEO_FEATURE_NORMALIZED = pd.DataFrame()
    
    for row, row_data in video_feature_rest_drop_na.iterrows():
        shape = vid_adj_fun.tuple_to_np(row_data[6:74])
        
        # Calculate the proportion
        dist_r = np.linalg.norm(shape[1-1] - shape[17-1])
        
        # Proportion
        if dist_r != 0:
            propotion = dist_base / dist_r
        else:
            propotion = 0
        
        # Scale
        new_shape = vid_adj_fun.transform_scale_updown(shape, propotion)
        
        # Mount the 68 Points in DataFrame
        count_id = 0
        for point in new_shape:
            row_data[str(count_id)] = str(tuple(point))
            count_id += 1
        new_row_df = pd.DataFrameMAIN_LIST_INFO
    
    # Output path
    base_path_folder_output = os.path.join(path_dir, output_measure_name)
    
    # Count Process
    print (str(ptr) + " of " + str(len(MAIN_LIST_INFO)) + ": " + base_path_folder_output)

    # Generate the OUTPUT CSV in the current folder
    VIDEO_FEATURE_NORMALIZED.to_csv(base_path_folder_output, na_rep='NaN')
    